In [127]:
!pip install selenium
from selenium import webdriver
import pandas as pd
from selenium.common import NoSuchElementException, StaleElementReferenceException
from matplotlib import style
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import re

print(plt.style.available)
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [152]:
driver = webdriver.Edge()
url = "https://esf.fang.com/house-a015277-b05/"
driver.get(url)

In [153]:
#def next_page(driver):
#    try:
#        next_page = WebDriverWait(driver, 10).until(
#            EC.presence_of_element_located((By.LINK_TEXT, "下一页"))
#        )
#        driver.execute_script("arguments[0].click();", next_page) 


In [154]:
i = 0
data = []
while i <= 2:
    houses = driver.find_elements(By.CSS_SELECTOR, 'dl.clearfix')
    for house in houses:
        title = house.find_element(By.CSS_SELECTOR, '.tit_shop').text
        info = house.find_element(By.CSS_SELECTOR, '.tel_shop').text
        price = house.find_element(By.CSS_SELECTOR, '.price_right').text
        item = {"名字":title,"信息":info,"价格":price}
        data.append(item)
    try:
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
    except NoSuchElementException:
        print("NoSuchElementException")
    i+=1

NoSuchElementException
NoSuchElementException


In [161]:
def house_price(house_price):
    if not isinstance(house_price, str): 
        return {'总价(万)': None, '单价(元/㎡)': None}

    total_price = re.search(r'(\d+(\.\d+)?)万', house_price) 
    unit_price = re.search(r'(\d+(\.\d+)?)元/㎡', house_price)

    return {
        '总价(万)': float(total_price.group(1)) if total_price else None,
        '单价(元/㎡)': float(unit_price.group(1)) if unit_price else None
    }

In [162]:
houses_data = []

for item in data:
    info = item["信息"].split("|")
    price = house_price(item["价格"])  
    layout = info[0] if len(info) > 0 else None
    area = float(info[1][:-2]) if len(info) > 1 and "㎡" in info[1] else None
    floor = info[2] if len(info) > 2 else None
    orientation = info[3] if len(info) > 3 else None
    year_built = int(info[4][:-3]) if len(info) > 4 and "年" in info[4] else None

    house_dict = {
        '名字': item.get("名字", "未知"),
        '布局': layout,
        '面积（㎡）': area,
        '楼层': floor,
        '朝向': orientation,
        '建成年份': year_built,
        **price  
    }

    houses_data.append(house_dict)  # 存入列表

df = pd.DataFrame(houses_data)

# 输出前几行检查
print(df.head())

                               名字     布局  面积（㎡）          楼层     朝向  建成年份  \
0       蓟门桥地铁央产大院双南向两居室精装交付满五年商品房  2室1厅   60.30  高层 （共16层）    东南向   1993   
1  北京邮电大学家属区 电梯房 视野开阔 采光好小户型 钢混结构  2室1厅   57.70  高层 （共16层）    西北向   1992   
2    小西天 北京邮电大学家属楼 正规朝南一居室 业主诚意出售  1室1厅   33.15   底层 （共5层）     南向   1985   
3          小西天 · 北京邮电大学 · 3室 · 1厅  3室1厅   93.49   中层 （共7层）     东向   2001   
4  蓟门桥 小西天 师大北 教科所 南向两居室 电梯房 看房方便  2室1厅   54.13  顶层 （共12层）     南向   1985   

   总价(万)   单价(元/㎡)  
0  598.0   99170.0  
1  530.0   91854.0  
2  350.0  105580.0  
3  805.0   86105.0  
4  415.0   76667.0  


In [163]:
df.to_csv(r"C:\Users\18910\Desktop\北太平庄房价.csv",encoding='utf_8_sig')